In [ ]:
import math
from multiprocessing import cpu_count, Pool

import ipywidgets as widgets

# Avoid non-compliant Type 3 fonts
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42  # pylint: disable=wrong-import-position

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from ipywidgets import interact
from tqdm.notebook import tqdm

import utils

In [ ]:
pd.set_option('display.max_rows', None)
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
logs_dir = utils.get_logs_dir()
eval_dir = utils.get_eval_dir()
env_names = ['small_empty', 'large_empty', 'large_columns', 'large_door', 'large_center']
step_size = 100

In [ ]:
# Load all runs
def get_cfgs():
    config_paths = sorted([x / 'config.yml' for x in logs_dir.iterdir() if x.is_dir()])
    with Pool(cpu_count()) as p:
        return list(tqdm(p.imap(utils.load_config, config_paths), total=len(config_paths)))
cfgs = get_cfgs()

In [ ]:
# def get_result_for_run(cfg):
#     metric_name = 'objects'
#     eval_path = eval_dir / f'{cfg.run_name}.npy'
#     data = np.load(eval_path, allow_pickle=True)
#     run_results = []
#     for episode in data:
#         run_results.append(episode[-1][metric_name])
#     return np.nanmean(run_results)

In [ ]:
# def get_all_results():
#     results = {}
#     with Pool(cpu_count()) as p:
#         result_per_run = list(tqdm(p.imap(get_result_for_run, cfgs), total=len(cfgs)))
#     for cfg, run_result in zip(cfgs, result_per_run):
#         if cfg.env_name not in results:
#             results[cfg.env_name] = {}
#         if cfg.experiment_name not in results[cfg.env_name]:
#             results[cfg.env_name][cfg.experiment_name] = []
#         results[cfg.env_name][cfg.experiment_name].append(run_result)

#     for env_results in results.values():
#         for experiment_name, values in env_results.items():
#             env_results[experiment_name] = f'{np.mean(values):.2f} ± {np.std(values):.2f}'
#     return results

In [ ]:
def extend_curves(curves, min_len=None):
    if len(curves) == 0:
        return curves
    max_length = max(len(curve) for curve in curves)
    if min_len is not None:
        max_length = max(max_length, min_len)
    for i, curve in enumerate(curves):
        curves[i] = np.pad(curve, (0, max_length - len(curve)), 'edge')
    return curves

In [ ]:
def get_curve_for_run(cfg):
    metric_name = 'objects'
    eval_path = eval_dir / f'{cfg.run_name}.npy'
    data = np.load(eval_path, allow_pickle=True)
    episode_curves = []
    for episode in data:
        values = np.array([step[metric_name] for step in episode])
        simulation_steps = np.array([step['simulation_steps'] for step in episode])
        x = np.arange(0, simulation_steps[-1] + step_size, step_size)
        xp, fp = simulation_steps, values
        episode_curves.append(np.floor(np.interp(x, xp, fp, left=0)))

    return np.mean(extend_curves(episode_curves), axis=0)

In [ ]:
def get_all_curves():
    curves = {}
    with Pool(cpu_count()) as p:
        curve_per_run = list(tqdm(p.imap(get_curve_for_run, cfgs), total=len(cfgs)))
    for cfg, run_curve in zip(cfgs, curve_per_run):
        if cfg.env_name not in curves:
            curves[cfg.env_name] = {}
        if cfg.experiment_name not in curves[cfg.env_name]:
            curves[cfg.env_name][cfg.experiment_name] = []
        curves[cfg.env_name][cfg.experiment_name].append(run_curve)
    return curves

In [ ]:
all_curves = get_all_curves()

In [ ]:
def get_all_cutoffs():
    cutoffs = {}
    reference_experiment_names = {
        'blowing_1-small_empty-multifreq_4',
        'blowing_1-large_empty-multifreq_4',
        'blowing_1-large_columns-multifreq_4',
        'blowing_1-large_door-multifreq_4',
        'blowing_1-large_center-multifreq_4',
    }
    for cfg in tqdm(cfgs):
        if cfg.experiment_name not in reference_experiment_names:
            continue
        if cfg.env_name not in cutoffs:
            cutoffs[cfg.env_name] = float('inf')

        # Find the time at which the last object was successfully cleaned up
        y_mean = np.mean(extend_curves(all_curves[cfg.env_name][cfg.experiment_name]), axis=0)
        this_len = np.searchsorted(y_mean > y_mean[-1] - 0.5, True)  # Subtract 0.5 since interpolated curves are continuous
        cutoffs[cfg.env_name] = min(cutoffs[cfg.env_name], this_len)
    return cutoffs

In [ ]:
all_cutoffs = get_all_cutoffs()

In [ ]:
def get_all_results():
    results = {}
    for cfg in tqdm(cfgs):
        cutoff = all_cutoffs[cfg.env_name]
        curves = extend_curves(all_curves[cfg.env_name][cfg.experiment_name], min_len=(cutoff + 1))
        objects = [curve[cutoff] for curve in curves]
        if cfg.env_name not in results:
            results[cfg.env_name] = {}
        results[cfg.env_name][cfg.experiment_name] = '{:.2f} ± {:.2f}'.format(np.mean(objects), np.std(objects))
    return results

In [ ]:
all_results = get_all_results()

In [ ]:
def show_table():
    def f(env_name):
        data = {'performance': all_results[env_name]}
        display(pd.DataFrame(data))

    env_name_dropdown = widgets.RadioButtons(options=env_names)
    interact(f, env_name=env_name_dropdown)

In [ ]:
show_table()

In [ ]:
def plot_curves(env_name, experiment_names,
                title=None, labels=None, legend_order=None, x_tick_step_size=None, x_lim_right=None, fontsize=26):
    # Plot curves
    for experiment_name in experiment_names:
        #plt.axvline(all_cutoffs[env_name] * step_size, linewidth=1, c='r')
        curves = extend_curves(all_curves[env_name][experiment_name])
        x = np.arange(0, (len(curves[0]) - 0.5) * step_size, step_size)
        y_mean, y_std = np.mean(curves, axis=0), np.std(curves, axis=0)
        plt.plot(x, y_mean, label=experiment_name if labels is None else labels[experiment_name])
        plt.fill_between(x, y_mean - y_std, y_mean + y_std, alpha=0.2)

    # Set up x-axis
    plt.xlabel('Simulation steps', fontsize=fontsize)
    if x_lim_right is not None and x_tick_step_size is not None:
        plt.xlim(0, x_lim_right)
        get_x_label = lambda x: 0 if x == 0 else f'{(x_tick_step_size // 1000) * x}k'
        x_labels = [get_x_label(i) for i in range(math.ceil((x_lim_right) / x_tick_step_size))]
        plt.xticks(range(0, x_lim_right, x_tick_step_size), labels=x_labels, fontsize=(fontsize - 8))
    else:
        plt.xlim(0)
        plt.xticks(fontsize=(fontsize - 8))

    # Set up y-axis
    plt.ylabel('Num objects', fontsize=fontsize)
    if env_name.startswith('large'):
        num_objects = 100
    else:
        num_objects = 50
    plt.ylim(0, num_objects)
    plt.yticks(range(0, num_objects + 1, max(1, num_objects // 5)), fontsize=(fontsize - 8))

    # Title
    if title is not None:
        plt.title(title, fontsize=(fontsize + 4), y=1.05)

    # Legend
    if legend_order is not None:
        handles, labels = plt.gca().get_legend_handles_labels()
        plt.legend([handles[i] for i in legend_order], [labels[i] for i in legend_order],
                   fontsize=(fontsize - 8), loc='lower right')
    else:
        plt.legend(fontsize=(fontsize - 8), loc='lower right')

In [ ]:
def show_curves():
    def f(env_name, experiment_names, save_to_pdf):
        if len(experiment_names) == 0:
            return
        plot_curves(env_name, experiment_names)
        if save_to_pdf:
            plt.savefig(f'curves-{env_name}.pdf', bbox_inches='tight')
        else:
            plt.show()

    env_name_radio = widgets.RadioButtons(options=env_names)
    experiment_names_select = widgets.SelectMultiple(layout=widgets.Layout(width='60%', height='150px'))
    save_toggle = widgets.ToggleButton(description='Save to PDF')
    def update_experiment_names_options(*_):
        matching_experiment_names = sorted(all_curves.get(env_name_radio.value, []))
        experiment_names_select.options = matching_experiment_names
        experiment_names_select.rows = len(matching_experiment_names)
        experiment_names_select.value = ()
    env_name_radio.observe(update_experiment_names_options)
    interact(f, env_name=env_name_radio,
             experiment_names=experiment_names_select,
             save_to_pdf=save_toggle)

In [ ]:
show_curves()

In [ ]:
def save_paper_plots():
    # Blowing vs. pushing
    plot_env_names = ['small_empty', 'large_empty', 'large_columns', 'large_door', 'large_center']
    titles = ['SmallEmpty', 'LargeEmpty', 'LargeColumns', 'LargeDoor', 'LargeCenter']
    x_tick_step_sizes = [20000, 45000, 60000, 80000, 50000]
    x_lim_rights = [96800, 250100, 267800, 392500, 242200]
    for env_name, title, x_tick_step_size, x_lim_right in zip(plot_env_names, titles, x_tick_step_sizes, x_lim_rights):
        plot_curves(
            env_name,
            [
                f'blowing_1-{env_name}-multifreq_4',
                f'blowing_1-{env_name}-singlefreq_4',
                f'pushing_1-{env_name}-singlefreq_4',
            ],
            title=title,
            labels={
                f'blowing_1-{env_name}-multifreq_4': 'Blowing (multi-freq)',
                f'blowing_1-{env_name}-singlefreq_4': 'Blowing (single-freq)',
                f'pushing_1-{env_name}-singlefreq_4': 'Pushing',
            },
            x_tick_step_size=x_tick_step_size,
            x_lim_right=x_lim_right,
            fontsize=40)
        if env_name == 'small_empty':
            plt.gca().yaxis.set_label_coords(-0.107, 0.5)
        plt.savefig(f'curves-{env_name}.pdf', bbox_inches='tight')
        plt.clf()

    # Blowing force
    plot_env_names = ['large_empty', 'large_center']
    titles = ['LargeEmpty', 'LargeCenter']
    x_tick_step_sizes = [70000, 70000]
    x_lim_rights = [400000, 400000]
    for env_name, title, x_tick_step_size, x_lim_right in zip(plot_env_names, titles, x_tick_step_sizes, x_lim_rights):
        legend_order = [2, 3, 1, 0] if env_name == 'large_center' else None
        plot_curves(
            env_name,
            [
                f'blowing_1-{env_name}-multifreq_4-blowforce_0.65',
                f'blowing_1-{env_name}-multifreq_4-blowforce_0.5',
                f'blowing_1-{env_name}-multifreq_4',
                f'blowing_1-{env_name}-multifreq_4-blowforce_0.2',
            ],
            title=title,
            labels={
                f'blowing_1-{env_name}-multifreq_4-blowforce_0.65': '0.65',
                f'blowing_1-{env_name}-multifreq_4-blowforce_0.5': '0.50',
                f'blowing_1-{env_name}-multifreq_4': '0.35',
                f'blowing_1-{env_name}-multifreq_4-blowforce_0.2': '0.20',
            },
            legend_order=legend_order,
            x_tick_step_size=x_tick_step_size,
            x_lim_right=x_lim_right,
            fontsize=40)
        plt.savefig(f'curves-{env_name}-blowing_force.pdf', bbox_inches='tight')
        plt.clf()

In [ ]:
save_paper_plots()